<a href="https://colab.research.google.com/github/coding-dojo-data-science/example-kaggle-lesson/blob/main/SOLUTIONS_4_18_Regression_Kaggle_Competition_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle Competition

## Regression: [Housing Prices Regression](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques)


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer

# Custom Functions

In [2]:
def explore_data(df):
  """display .info(), num duplicates, num missing values, categories in 
  categorical columns, and descriptive statistics of numeric columns
  """
  
  # Explore Columns
  print('Columns:')
  display(df.info())
  print('\n')

  # Find duplicates
  print(f'{df.duplicated().sum()} Duplicates Found \n')

  # Check for missing values
  print('Missing Values')
  display(df.isna().sum())
  print('\n')

  # Examine categories
  for col in df.select_dtypes(include='object').columns:
    print(col)
    display(df[col].value_counts())
    print('\n')

  # Show Summary Statistics
  display(df.describe(include='all'))

def split_data(df, target, random_state=42):
  """Split df features and target and perform train/test split.
  returns 4 values: X_train, X_test, y_train, y_test.
  optional argument: random_state
  """

  # Split columns
  X = df.drop(target, axis=1)
  y = df[target]

  # Return train/test split (4 values)
  return train_test_split(X, y, random_state=random_state)

def evaluate_regression(model, X, y, index=[0]):
  """Evaluates a model on a given set of features and target.
  model should be already fitted.
  Returns a dataframe with metrics as columns and one row
  optional argument: index value for the row.  default: 0
  """

  preds = model.predict(X)
  scores = {'R2': r2_score(y, preds),
            'MAE': mean_absolute_error(y, preds),
            'MSE': mean_squared_error(y, preds),
            'RMSE': np.sqrt(mean_squared_error(y, preds))}
  scores = pd.DataFrame(scores, index=index)
  
  return scores

def try_regmodel(model, X_train, X_test, y_train, y_test, param_grid=None):
  """fits and evaluates a model.  Returns a fitted model and a dataframe of scores
  for both train and test datasets.
  If a param_grid is provided, will fit and return a GridSearchCV object 
  using that grid.
  """

  if param_grid:
    model = GridSearchCV(model, param_grid)
  model.fit(X_train, y_train)

  train_scores = evaluate_regression(model, X_train, y_train, index=['Train'])
  test_scores = evaluate_regression(model, X_test, y_test, index=['Test'])

  scores = pd.concat([train_scores, test_scores], axis=0)
  return model, scores

# Import Data

In [3]:
train_path = '/content/train.csv'
train_df = pd.read_csv(train_path)

train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
# Lower case columns for ease
train_df.columns = train_df.columns.str.lower()

# EDA and Cleaning

In [5]:
explore_data(train_df)

Columns:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1460 non-null   int64  
 1   mssubclass     1460 non-null   int64  
 2   mszoning       1460 non-null   object 
 3   lotfrontage    1201 non-null   float64
 4   lotarea        1460 non-null   int64  
 5   street         1460 non-null   object 
 6   alley          91 non-null     object 
 7   lotshape       1460 non-null   object 
 8   landcontour    1460 non-null   object 
 9   utilities      1460 non-null   object 
 10  lotconfig      1460 non-null   object 
 11  landslope      1460 non-null   object 
 12  neighborhood   1460 non-null   object 
 13  condition1     1460 non-null   object 
 14  condition2     1460 non-null   object 
 15  bldgtype       1460 non-null   object 
 16  housestyle     1460 non-null   object 
 17  overallqual    1460 non-null   int64  
 18 

None



0 Duplicates Found 

Missing Values


id                 0
mssubclass         0
mszoning           0
lotfrontage      259
lotarea            0
                ... 
mosold             0
yrsold             0
saletype           0
salecondition      0
saleprice          0
Length: 81, dtype: int64



mszoning


...



saletype


WD       1267
New       122
COD        43
ConLD       9
ConLI       5
ConLw       5
CWD         4
Oth         3
Con         2
Name: saletype, dtype: int64



salecondition


Normal     1198
Partial     125
Abnorml     101
Family       20
Alloca       12
AdjLand       4
Name: salecondition, dtype: int64

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
count,1460.000000,1460.000000,1460,1201.000000,1460.000000,1460,91,1460,1460,1460,...,1460.000000,7,281,54,1460.000000,1460.000000,1460.000000,1460,1460,1460.000000
unique,NaN,NaN,5,NaN,NaN,2,2,4,4,2,...,NaN,3,4,4,NaN,NaN,NaN,9,6,NaN
top,NaN,NaN,RL,NaN,NaN,Pave,Grvl,Reg,Lvl,AllPub,...,NaN,Gd,MnPrv,Shed,NaN,NaN,NaN,WD,Normal,NaN
freq,NaN,NaN,1151,NaN,NaN,1454,50,925,1311,1459,...,NaN,3,157,49,NaN,NaN,NaN,1267,1198,NaN
mean,730.500000,56.897260,NaN,70.049958,10516.828082,NaN,NaN,NaN,NaN,NaN,...,2.758904,NaN,NaN,NaN,43.489041,6.321918,2007.815753,NaN,NaN,180921.195890
std,421.610009,42.300571,NaN,24.284752,9981.264932,NaN,NaN,NaN,NaN,NaN,...,40.177307,NaN,NaN,NaN,496.123024,2.703626,1.328095,NaN,NaN,79442.502883
min,1.000000,20.000000,NaN,21.000000,1300.000000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,1.000000,2006.000000,NaN,NaN,34900.000000
25%,365.750000,20.000000,NaN,59.000000,7553.500000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,5.000000,2007.000000,NaN,NaN,129975.000000
50%,730.500000,50.000000,NaN,69.000000,9478.500000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,6.000000,2008.000000,NaN,NaN,163000.000000
75%,1095.250000,70.000000,NaN,80.000000,11601.500000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,8.000000,2009.000000,NaN,NaN,214000.000000


In [6]:
# Drop columns missing more than 40% of data
na_thresh = .6
train_df = train_df.dropna(axis=1, thresh=train_df.shape[0] * na_thresh)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 76 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1460 non-null   int64  
 1   mssubclass     1460 non-null   int64  
 2   mszoning       1460 non-null   object 
 3   lotfrontage    1201 non-null   float64
 4   lotarea        1460 non-null   int64  
 5   street         1460 non-null   object 
 6   lotshape       1460 non-null   object 
 7   landcontour    1460 non-null   object 
 8   utilities      1460 non-null   object 
 9   lotconfig      1460 non-null   object 
 10  landslope      1460 non-null   object 
 11  neighborhood   1460 non-null   object 
 12  condition1     1460 non-null   object 
 13  condition2     1460 non-null   object 
 14  bldgtype       1460 non-null   object 
 15  housestyle     1460 non-null   object 
 16  overallqual    1460 non-null   int64  
 17  overallcond    1460 non-null   int64  
 18  yearbuil

In [7]:
# drop columns unlikely to be helpful in prediction
drop_cols = ['id','street','utilities','condition2','roofmatl']
train_df = train_df.drop(columns=drop_cols)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 71 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   mssubclass     1460 non-null   int64  
 1   mszoning       1460 non-null   object 
 2   lotfrontage    1201 non-null   float64
 3   lotarea        1460 non-null   int64  
 4   lotshape       1460 non-null   object 
 5   landcontour    1460 non-null   object 
 6   lotconfig      1460 non-null   object 
 7   landslope      1460 non-null   object 
 8   neighborhood   1460 non-null   object 
 9   condition1     1460 non-null   object 
 10  bldgtype       1460 non-null   object 
 11  housestyle     1460 non-null   object 
 12  overallqual    1460 non-null   int64  
 13  overallcond    1460 non-null   int64  
 14  yearbuilt      1460 non-null   int64  
 15  yearremodadd   1460 non-null   int64  
 16  roofstyle      1460 non-null   object 
 17  exterior1st    1460 non-null   object 
 18  exterior

# Modeling

In [8]:
X_train, X_test, y_train, y_test = split_data(train_df, 'saleprice')
X_train.shape

(1095, 70)

## Preprocessor

In [9]:
cat_sel = make_column_selector(dtype_include='object')
num_sel = make_column_selector(dtype_include='number')

scaler = StandardScaler()
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
missing_imputer = SimpleImputer(strategy='constant', fill_value='missing')
mean_imputer = SimpleImputer(strategy='mean')

In [10]:
cat_pipe = make_pipeline(missing_imputer, ohe)
num_pipe = make_pipeline(mean_imputer, scaler)

preprocessor = make_column_transformer((cat_pipe, cat_sel),
                                       (num_pipe, num_sel))

np.isnan(preprocessor.fit_transform(X_train)).sum()

0

## Model 1: Linear Regression

In [11]:
lin_reg_pipe = make_pipeline(preprocessor, LinearRegression())

data = (X_train, X_test, y_train, y_test)
lin_model, linreg_scores = try_regmodel(lin_reg_pipe, X_train, X_test, y_train, y_test)
display(linreg_scores)

,R2,MAE,MSE,RMSE
Train,9.003669e-01,1.568456e+04,6.049170e+08,2.459506e+04
Test,-1.279652e+21,1.786414e+14,8.964360e+30,2.994054e+15


### Turning Linear Regression: L1 (Lasso)

In [12]:
lasso_pipe = make_pipeline(preprocessor, Lasso())

lasso_params = {'lasso__alpha':[.01, .1, 1, 10, 100]}

lasso, lasso_scores = try_regmodel(lasso_pipe, X_train, X_test, y_train, y_test, param_grid = lasso_params)

display(lasso_scores)
lasso.best_estimator_.get_params()['lasso']

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.874e+10, tolerance: 4.973e+08
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.652e+10, tolerance: 5.586e+08
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisa

,R2,MAE,MSE,RMSE
Train,0.888363,16000.281071,6.777995e+08,26034.582433
Test,0.877497,18596.726221,8.581702e+08,29294.542098


Lasso(alpha=100)

# Tuning Linear Regression: L2 (Ridge)

In [13]:
ridge_pipe = make_pipeline(preprocessor, Ridge())

ridge_params = {'ridge__alpha':[.01, .1, 1, 10, 100]}

ridge, ridge_scores = try_regmodel(ridge_pipe, X_train, X_test, y_train, y_test, param_grid = ridge_params)

display(ridge_scores)
ridge.best_estimator_.get_params()['ridge']

,R2,MAE,MSE,RMSE
Train,0.862076,16589.954914,8.373989e+08,28937.845676
Test,0.866679,18266.965693,9.339515e+08,30560.619361


Ridge(alpha=100)

## Model 2: XGBoost

In [14]:
%%time
from xgboost import XGBRegressor

xgb_pipe = make_pipeline(preprocessor, XGBRegressor(n_jobs=-1,
                                                    objective='reg:squarederror'))

xgb, xgb_score = try_regmodel(xgb_pipe, X_train, X_test, y_train, y_test)

display(xgb_score)


,R2,MAE,MSE,RMSE
Train,0.966468,10164.818415,2.035866e+08,14268.377636
Test,0.905213,16280.972389,6.640119e+08,25768.428531


CPU times: user 3.17 s, sys: 13.4 ms, total: 3.19 s
Wall time: 3.45 s


In [15]:
xgb_params = {'xgbregressor__max_depth': range(1, 10)
              }

xgb_gs, xgb_score = try_regmodel(xgb_pipe, X_train, X_test, y_train, y_test, param_grid=xgb_params)

display(xgb_score)
xgb_gs.best_estimator_.named_steps['xgbregressor'].get_params()

,R2,MAE,MSE,RMSE
Train,0.994834,4117.699704,3.136234e+07,5600.208833
Test,0.912755,15638.016995,6.111753e+08,24721.959829


{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'importance_type': 'gain',
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 6,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'n_jobs': -1,
 'nthread': None,
 'objective': 'reg:squarederror',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': None,
 'subsample': 1,
 'verbosity': 1}

# Prepare Data for Submission

## Load the Submission Test File

In [16]:
test_path = '/content/test.csv'
test_df = pd.read_csv(test_path)

# lower case columns

test_df.columns = test_df.columns.str.lower()
test_df.columns

Index(['id', 'mssubclass', 'mszoning', 'lotfrontage', 'lotarea', 'street',
       'alley', 'lotshape', 'landcontour', 'utilities', 'lotconfig',
       'landslope', 'neighborhood', 'condition1', 'condition2', 'bldgtype',
       'housestyle', 'overallqual', 'overallcond', 'yearbuilt', 'yearremodadd',
       'roofstyle', 'roofmatl', 'exterior1st', 'exterior2nd', 'masvnrtype',
       'masvnrarea', 'exterqual', 'extercond', 'foundation', 'bsmtqual',
       'bsmtcond', 'bsmtexposure', 'bsmtfintype1', 'bsmtfinsf1',
       'bsmtfintype2', 'bsmtfinsf2', 'bsmtunfsf', 'totalbsmtsf', 'heating',
       'heatingqc', 'centralair', 'electrical', '1stflrsf', '2ndflrsf',
       'lowqualfinsf', 'grlivarea', 'bsmtfullbath', 'bsmthalfbath', 'fullbath',
       'halfbath', 'bedroomabvgr', 'kitchenabvgr', 'kitchenqual',
       'totrmsabvgrd', 'functional', 'fireplaces', 'fireplacequ', 'garagetype',
       'garageyrblt', 'garagefinish', 'garagecars', 'garagearea', 'garagequal',
       'garagecond', 'paveddrive

In [17]:
# Isolate test features
test_features = test_df[X_train.columns]
test_features.head()

,mssubclass,mszoning,lotfrontage,lotarea,lotshape,landcontour,lotconfig,landslope,neighborhood,condition1,...,openporchsf,enclosedporch,3ssnporch,screenporch,poolarea,miscval,mosold,yrsold,saletype,salecondition
0,20,RH,80.0,11622,Reg,Lvl,Inside,Gtl,NAmes,Feedr,...,0,0,0,120,0,0,6,2010,WD,Normal
1,20,RL,81.0,14267,IR1,Lvl,Corner,Gtl,NAmes,Norm,...,36,0,0,0,0,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,IR1,Lvl,Inside,Gtl,Gilbert,Norm,...,34,0,0,0,0,0,3,2010,WD,Normal
3,60,RL,78.0,9978,IR1,Lvl,Inside,Gtl,Gilbert,Norm,...,36,0,0,0,0,0,6,2010,WD,Normal
4,120,RL,43.0,5005,IR1,HLS,Inside,Gtl,StoneBr,Norm,...,82,0,0,144,0,0,1,2010,WD,Normal


In [18]:
# Create submission df
submission_df = test_df[['id']]
submission_df.head()

,id
0,1461
1,1462
2,1463
3,1464
4,1465


## Process and predict sale price features using the best model


In [19]:
# Predict SalePrice
submission_preds = xgb.predict(test_features)
submission_df['SalePrice'] = submission_preds
submission_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,id,SalePrice
0,1461,125465.570312
1,1462,158691.750000
2,1463,170905.093750
3,1464,184529.140625
4,1465,200474.093750


# Compare my submission df with sample submission df

In [20]:
# Examine sample submission
sample = pd.read_csv('/content/sample_submission.csv')
sample.head()

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977


## Fix the column names

In [21]:
submission_df.columns=['Id','SalePrice']
submission_df.head()

,Id,SalePrice
0,1461,125465.570312
1,1462,158691.750000
2,1463,170905.093750
3,1464,184529.140625
4,1465,200474.093750


# Save my prediction submission to upload to Kaggle

In [22]:
submission_df.to_csv('/content/submission.csv')